In [1]:
from consensus.PBFT_Simulator import PBFT_Simulator
from consensus.QPBFT_Simulator import QPBFT_Simulator
from consensus.Proposed_Simulator import Proposed_Simulator
import time

In [2]:
consensus = PBFT_Simulator(10, 2)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} sec")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 1, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 2, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 3, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 4, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 5, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 6, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 7, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 8, Faulty => False, Send_messages => ('Hello world 0', 'pre-prepare', 8), Receive_messages => [('Hello world 0', 'pre-pr

In [3]:
del consensus

In [10]:
consensus = QPBFT_Simulator(6, 4, 2)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} s")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Role =>  Manager, Faulty =>  False, send_message_log => ('Hello world 0', 'prepare', 0)    , messages_log => [('Hello world 0', 'prepare', 0, '5310eca6643a80e0bafcc136e6c89b61921827e21289714b66752fb1d9425448')]
IdUser => 1, Role =>  Manager, Faulty =>  False, send_message_log => ''                                 , messages_log => []
IdUser => 2, Role =>  Manager, Faulty =>  False, send_message_log => ''                                 , messages_log => []
IdUser => 3, Role =>  Manager, Faulty =>  False, send_message_log => ''                                 , messages_log => []
IdUser => 4, Role =>  Manager, Faulty =>  False, send_message_log => ''                                 , messages_log => []
IdUser => 5, Role =>  Manager, Faulty =>   True, send_message_log => ''                                 , messages_log => []
IdUser => 6, Role =>    Voter, Faulty =>   True, send_message_log => ''                                 , messages_log => []
IdUser => 7,

In [5]:
del consensus

In [6]:
consensus = Proposed_Simulator(6, 4, 5, 3)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} s")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 5, Role =>  Master, send_message_log => ''                                 , messages_log => []
IdUser => 9, Role =>   Slave, send_message_log => ''                                 , messages_log => [('Hello world 0', 'request', -1)]
IdUser => 7, Role =>   Slave, send_message_log => ''                                 , messages_log => []
IdUser => 1, Role =>  Master, send_message_log => ''                                 , messages_log => []
IdUser => 0, Role =>  Master, send_message_log => ''                                 , messages_log => []
------------------------------
Prepare Internal Phase
IdUser => 5, Role =>  Master, send_message_log => ''                                 , messages_log => [('Hello world 0', 'prepare', 9)]
IdUser => 9, Role =>   Slave, send_message_log => ('Hello world 0', 'prepare', 9)    , messages_log => [('Hello world 0', 'request', -1)]
IdUser => 7, Role =>   Slave, send_message_log => ''                                 , messages

In [7]:
del consensus